# Inspecting the e-SNLI Dataset

## Checking the transformed DeepA2-ESNLI dataset

In [41]:
from deepa2.config import data_dir
from datasets import Dataset

In [47]:
path = (data_dir / "processed" / "esnli" / "train" / "train.parquet")
path = path.resolve()
path.exists()

True

In [49]:
da2_dataset = Dataset.from_parquet(str(path))

Using custom data configuration default-3f29edf1a53ed96a


100%|██████████| 1/1 [00:00<00:00, 569.80it/s]


Dataset parquet downloaded and prepared to /Users/ggbetz/.cache/huggingface/datasets/parquet/default-3f29edf1a53ed96a/0.0.0/1638526fd0e8d960534e2155dc54fdff8dce73851f21f031d2fb9c2cf757c121. Subsequent calls will reuse this data.


In [50]:
da2_dataset

Dataset({
    features: ['argdown_reconstruction', 'source_text', 'conclusion', 'conclusion_formalized', 'conjectures', 'context', 'distractors', 'entity_placeholders', 'erroneous_argdown', 'gist', 'intermediary_conclusion', 'intermediary_conclusion_formalized', 'metadata', 'misc_placeholders', 'predicate_placeholders', 'premises', 'premises_formalized', 'reasons', 'source_paraphrase', 'title'],
    num_rows: 31200
})

In [51]:
da2_dataset[50]

{'argdown_reconstruction': '(1) # 6 tries her best to help her team to victory.\n(2) if a sports arena is empty then it is wrong that # 6 tries her best to help her team to victory.\n--\nwith modus tollens from (1) (2)\n--\n(3) it is wrong that a sports arena is empty.',
 'source_text': 'The team cannot be helped to victory if the arena is empty. if she is helping her team then it is inferred she is playing a team sport it is wrong that a sports arena is empty. # 6 tries her best to help her team to victory.',
 'conclusion': [{'explicit': True,
   'ref_reco': 3,
   'text': 'it is wrong that a sports arena is empty.'}],
 'conclusion_formalized': [{'form': '¬{q}', 'ref_reco': 3}],
 'conjectures': [{'ref_reco': 3,
   'starts_at': 135,
   'text': 'it is wrong that a sports arena is empty.'}],
 'context': None,
 'distractors': ['if she is helping her team then it is inferred she is playing a team sport'],
 'entity_placeholders': None,
 'erroneous_argdown': '(1) # 6 tries her best to help he